### Problem Description
In this problem, we have six end distributors, each with a known demand for a product. Distributor demand can be satisfied from a set of four depots, or directly from a set of two factories. Each depot can support a maximum volume of product moving through it, and each factory can produce a maximum amount of product. There are known costs associated with transporting the product, from a factory to a depot, from a depot to a distributor, or from a factory directly to a distributor.

**Sets**
- Factories = {NewDelhi, Vishakhapatnam}
- Depots = {Ahmedabad, Kolkata, Coimbatore, Nagpur}
- Distributors = {C1, C2, C3, C4, C5, C6}
- Cities = Factories ∪ Depots ∪ Distributors

**Parameters**
- cost: Cost of shipping one ton from source  s  to destination  t.
- supply: Maximum possible supply from factory  f  (in tons).
- throughput: Maximum possible flow through depot  d  (in tons).
- demand: Demand for goods at distributor  c  (in tons).

**Decision Variables**
- flow: Quantity of goods (in tons) that is shipped from source  s  to destionation  t.

**Objective Function**
- Cost: Minimize total shipping costs.
**Minimize = cost ∗ flow**
 
**Constraints**
Factory output: Flow of goods from a factory must respect maximum capacity.
- flow ≤ supply
 
Distributor demand: Flow of goods must meet distributor demand.
- flow = demand
 
Depot flow: Flow into a depot equals flow out of the depot.
- flow = flow (from source to depot and depot to destination)
 
Depot capacity: Flow into a depot must respect depot capacity.
- flow ≤ throughput

In [1]:
from pulp import *
import pandas as pd

In [2]:
# Create dictionaries to capture factory supply limits, depot throughput limits, and distributor demand.
supply = dict({'NewDelhi': 150000,
               'Vishakhapatnam': 200000})

throughput = dict({'Ahmedabad': 70000,
                   'Kolkata': 50000,
                   'Coimbatore': 100000,
                   'Nagpur': 40000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

In [3]:
# Create a dictionary to capture shipping costs.
cost = dict({'NewDelhi': {'Ahmedabad' : 0.5, 'Kolkata': 0.5, 'Coimbatore': 1.0, 'Nagpur': 0.2, 'C1': 1.0, 'C3': 1.5,
                          'C4': 2.0, 'C6': 1.0},
             'Vishakhapatnam': {'Kolkata': 0.3, 'Coimbatore': 0.5, 'Nagpur': 0.2, 'C1': 2.0},
             'Ahmedabad': {'C2': 1.5, 'C3': 0.5, 'C5': 1.5, 'C6': 1.0},
             'Kolkata': {'C1': 1.0, 'C2': 0.5, 'C3': 0.5, 'C4': 1.0, 'C5': 0.5},
             'Coimbatore': {'C2': 1.5, 'C3': 2.0, 'C5': 0.5, 'C6': 1.5},
             'Nagpur': {'C3': 0.2, 'C4': 1.5, 'C5': 0.5, 'C6': 1.5}})
cost

{'NewDelhi': {'Ahmedabad': 0.5,
  'Kolkata': 0.5,
  'Coimbatore': 1.0,
  'Nagpur': 0.2,
  'C1': 1.0,
  'C3': 1.5,
  'C4': 2.0,
  'C6': 1.0},
 'Vishakhapatnam': {'Kolkata': 0.3,
  'Coimbatore': 0.5,
  'Nagpur': 0.2,
  'C1': 2.0},
 'Ahmedabad': {'C2': 1.5, 'C3': 0.5, 'C5': 1.5, 'C6': 1.0},
 'Kolkata': {'C1': 1.0, 'C2': 0.5, 'C3': 0.5, 'C4': 1.0, 'C5': 0.5},
 'Coimbatore': {'C2': 1.5, 'C3': 2.0, 'C5': 0.5, 'C6': 1.5},
 'Nagpur': {'C3': 0.2, 'C4': 1.5, 'C5': 0.5, 'C6': 1.5}}

In [4]:
sources = []
destinations = []
for i in cost.keys():
    for j in cost[i]:
        sources.append(i)
        destinations.append(j)
possibleFlows = list(zip(sources, destinations))
possibleFlows

[('NewDelhi', 'Ahmedabad'),
 ('NewDelhi', 'Kolkata'),
 ('NewDelhi', 'Coimbatore'),
 ('NewDelhi', 'Nagpur'),
 ('NewDelhi', 'C1'),
 ('NewDelhi', 'C3'),
 ('NewDelhi', 'C4'),
 ('NewDelhi', 'C6'),
 ('Vishakhapatnam', 'Kolkata'),
 ('Vishakhapatnam', 'Coimbatore'),
 ('Vishakhapatnam', 'Nagpur'),
 ('Vishakhapatnam', 'C1'),
 ('Ahmedabad', 'C2'),
 ('Ahmedabad', 'C3'),
 ('Ahmedabad', 'C5'),
 ('Ahmedabad', 'C6'),
 ('Kolkata', 'C1'),
 ('Kolkata', 'C2'),
 ('Kolkata', 'C3'),
 ('Kolkata', 'C4'),
 ('Kolkata', 'C5'),
 ('Coimbatore', 'C2'),
 ('Coimbatore', 'C3'),
 ('Coimbatore', 'C5'),
 ('Coimbatore', 'C6'),
 ('Nagpur', 'C3'),
 ('Nagpur', 'C4'),
 ('Nagpur', 'C5'),
 ('Nagpur', 'C6')]

In [5]:
# Model Definition
model = LpProblem("Shipping_Cost_Optimization", LpMinimize)

In [6]:
plant, depot, distributor = supply.keys(), throughput.keys(), demand.keys()
plant, depot, distributor

(dict_keys(['NewDelhi', 'Vishakhapatnam']),
 dict_keys(['Ahmedabad', 'Kolkata', 'Coimbatore', 'Nagpur']),
 dict_keys(['C1', 'C2', 'C3', 'C4', 'C5', 'C6']))

In [8]:
# Decision Variable
flowVar = LpVariable.dicts(name = 'Flow', 
                           indexs = [(src+'_'+dest) for src in cost.keys() for dest in cost[src]], 
                           lowBound = 0, cat = LpContinuous)
flowVar

{'NewDelhi_Ahmedabad': Flow_NewDelhi_Ahmedabad,
 'NewDelhi_Kolkata': Flow_NewDelhi_Kolkata,
 'NewDelhi_Coimbatore': Flow_NewDelhi_Coimbatore,
 'NewDelhi_Nagpur': Flow_NewDelhi_Nagpur,
 'NewDelhi_C1': Flow_NewDelhi_C1,
 'NewDelhi_C3': Flow_NewDelhi_C3,
 'NewDelhi_C4': Flow_NewDelhi_C4,
 'NewDelhi_C6': Flow_NewDelhi_C6,
 'Vishakhapatnam_Kolkata': Flow_Vishakhapatnam_Kolkata,
 'Vishakhapatnam_Coimbatore': Flow_Vishakhapatnam_Coimbatore,
 'Vishakhapatnam_Nagpur': Flow_Vishakhapatnam_Nagpur,
 'Vishakhapatnam_C1': Flow_Vishakhapatnam_C1,
 'Ahmedabad_C2': Flow_Ahmedabad_C2,
 'Ahmedabad_C3': Flow_Ahmedabad_C3,
 'Ahmedabad_C5': Flow_Ahmedabad_C5,
 'Ahmedabad_C6': Flow_Ahmedabad_C6,
 'Kolkata_C1': Flow_Kolkata_C1,
 'Kolkata_C2': Flow_Kolkata_C2,
 'Kolkata_C3': Flow_Kolkata_C3,
 'Kolkata_C4': Flow_Kolkata_C4,
 'Kolkata_C5': Flow_Kolkata_C5,
 'Coimbatore_C2': Flow_Coimbatore_C2,
 'Coimbatore_C3': Flow_Coimbatore_C3,
 'Coimbatore_C5': Flow_Coimbatore_C5,
 'Coimbatore_C6': Flow_Coimbatore_C6,
 'Nagp

In [9]:
# Factory Output constraint
for src in plant:
    model.addConstraint(LpConstraint(e = lpSum(flowVar[src+'_'+dest] for dest in cost[src]),
                                     sense = LpConstraintLE,
                                     name = 'Factory_Capacity_From_' + src,
                                     rhs = supply[src]))

In [10]:
# Customer Demand constraint
for dest in distributor:
    model.addConstraint(LpConstraint(
        e = lpSum(flowVar[possibleFlows[i][0] + '_' + possibleFlows[i][1]] for i in range(0, len(possibleFlows)) if possibleFlows[i][1] == dest),
        sense = LpConstraintEQ,
        name = 'Customer_Demand_At_' + dest,
        rhs = demand[dest]))

In [11]:
# Depot throughput constraint
for src in depot:
    model.addConstraint(LpConstraint(e = lpSum(flowVar[src+'_'+dest] for dest in cost[src]),
                                     sense = LpConstraintLE,
                                     name = 'Depot_Capacity_From_' + src,
                                     rhs = throughput[src]))

In [12]:
# Depot capacity constraint
for src in depot:
    model.addConstraint(LpConstraint(
        e = lpSum(flowVar[possibleFlows[i][0] + '_' + possibleFlows[i][1]] for i in range(0, len(possibleFlows)) if possibleFlows[i][1] == src) - 
        lpSum(flowVar[src + '_' + dest] for dest in cost[src]),
        sense = LpConstraintEQ,
        name = 'Depot_Throughput_At_' + src,
        rhs = 0))

In [14]:
# Model Objective
objective = lpSum(lpSum(cost[src][dest] * flowVar[src + '_' + dest] for dest in cost[src]) for src in cost.keys())
objective
model.setObjective(objective)
model

Shipping_Cost_Optimization:
MINIMIZE
1.5*Flow_Ahmedabad_C2 + 0.5*Flow_Ahmedabad_C3 + 1.5*Flow_Ahmedabad_C5 + 1.0*Flow_Ahmedabad_C6 + 1.5*Flow_Coimbatore_C2 + 2.0*Flow_Coimbatore_C3 + 0.5*Flow_Coimbatore_C5 + 1.5*Flow_Coimbatore_C6 + 1.0*Flow_Kolkata_C1 + 0.5*Flow_Kolkata_C2 + 0.5*Flow_Kolkata_C3 + 1.0*Flow_Kolkata_C4 + 0.5*Flow_Kolkata_C5 + 0.2*Flow_Nagpur_C3 + 1.5*Flow_Nagpur_C4 + 0.5*Flow_Nagpur_C5 + 1.5*Flow_Nagpur_C6 + 0.5*Flow_NewDelhi_Ahmedabad + 1.0*Flow_NewDelhi_C1 + 1.5*Flow_NewDelhi_C3 + 2.0*Flow_NewDelhi_C4 + 1.0*Flow_NewDelhi_C6 + 1.0*Flow_NewDelhi_Coimbatore + 0.5*Flow_NewDelhi_Kolkata + 0.2*Flow_NewDelhi_Nagpur + 2.0*Flow_Vishakhapatnam_C1 + 0.5*Flow_Vishakhapatnam_Coimbatore + 0.3*Flow_Vishakhapatnam_Kolkata + 0.2*Flow_Vishakhapatnam_Nagpur + 0.0
SUBJECT TO
Factory_Capacity_From_NewDelhi: Flow_NewDelhi_Ahmedabad + Flow_NewDelhi_C1
 + Flow_NewDelhi_C3 + Flow_NewDelhi_C4 + Flow_NewDelhi_C6
 + Flow_NewDelhi_Coimbatore + Flow_NewDelhi_Kolkata + Flow_NewDelhi_Nagpur
 <= 15000

In [15]:
model.solve()
print("Model Status = ", LpStatus[model.status])

Model Status =  Optimal


In [18]:
# Each of the variables is printed with it's resolved optimum value
df = pd.DataFrame()
variableName = []
variableValue = []
for v in model.variables():
#     print(v.name, "=", v.varValue)
    variableName.append(v.name)
    variableValue.append(v.varValue)

df['Variables'] = variableName
df['Variable_Values'] = variableValue
# The optimised objective function value is printed to the screen    
print("Product demand from all of our customers can be satisfied for a total cost of ", value(model.objective))

Product demand from all of our customers can be satisfied for a total cost of  198500.0


In [19]:
print("The optimal plan is as follows:")
df[df.Variable_Values > 0].reset_index(drop = True)

The optimal plan is as follows:


,Variables,Variable_Values
0,Flow_Coimbatore_C5,55000.0
1,Flow_Kolkata_C2,10000.0
2,Flow_Kolkata_C4,35000.0
3,Flow_Kolkata_C5,5000.0
4,Flow_Nagpur_C3,40000.0
5,Flow_NewDelhi_C1,50000.0
6,Flow_NewDelhi_C6,20000.0
7,Flow_NewDelhi_Nagpur,40000.0
8,Flow_Vishakhapatnam_Coimbatore,55000.0
9,Flow_Vishakhapatnam_Kolkata,50000.0
